## Case Study #1 - Danny's Diner

### Problem Statement
Danny wants to use the data to answer a few simple questions about his customers, especially about their visiting patterns, how much money they’ve spent and also which menu items are their favourite. Having this deeper connection with his customers will help him deliver a better and more personalised experience for his loyal customers.

He plans on using these insights to help him decide whether he should expand the existing customer loyalty program - additionally he needs help to generate some basic datasets so his team can easily inspect the data without needing to use SQL.

In [25]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('dannys_diner.sqlite3') 

### Question 1: How much did each customer spend?

In [26]:
sql = ('''
       SELECT customer_id, SUM(price) AS spend
       FROM sales
       INNER JOIN menu
         ON sales.product_id = menu.product_id
       GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  spend
0           A     76
1           B     74
2           C     36


### Question 2: How many days has each customer visited the restaurant?

In [27]:
sql = ('''
       SELECT customer_id, COUNT(DISTINCT order_date) AS visit_days
       FROM sales
       GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  visit_days
0           A           4
1           B           6
2           C           2


### Question 3: What was the first item from the menu purchased by each customer?

In [28]:
sql = ('''
       WITH product_rn AS (
	        SELECT *,
		           ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY order_date) as row_num
            FROM sales)

       SELECT customer_id, product_name as first_product
       FROM product_rn
       INNER JOIN menu
         ON product_rn.product_id = menu.product_id
        WHERE row_num = 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id first_product
0           A         sushi
1           B         curry
2           C         ramen


### Question 4: What is the most purchased item on the menu and how many times was it purchased by all customers?

In [29]:
sql = ('''
       SELECT product_name, COUNT(*) AS purchases
       FROM sales
       INNER JOIN menu
         ON sales.product_id = menu.product_id
       GROUP BY product_name
       ORDER BY purchases DESC
       LIMIT 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  product_name  purchases
0        ramen          8


### Question 5: Which item was the most popular for each customer?

In [30]:
sql = ('''
       WITH most_ordered AS(
         SELECT customer_id,
                product_id,
                COUNT(*) AS purchases,
                RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(*) DESC) AS rank
	     FROM sales
	     GROUP BY customer_id, product_id)

    SELECT customer_id,
        product_name,
        purchases
    FROM most_ordered
    INNER JOIN menu
        ON most_ordered.product_id = menu.product_id
    WHERE rank = 1
    ORDER BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id product_name  purchases
0           A        ramen          3
1           B        sushi          2
2           B        curry          2
3           B        ramen          2
4           C        ramen          3


### Question 6: Which item was purchased first by the customer after they became a member?

In [31]:
sql = ('''
       WITH after_join 
            AS (SELECT *,
                    RANK() OVER(PARTITION BY sales.customer_id ORDER BY order_date) AS ranking
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date >= join_date
                )

        SELECT customer_id,
               product_name
        FROM after_join
        WHERE ranking = 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id product_name
0           A        curry
1           B        sushi


### Question 7: Which item was purchased just before the customer became a member?
Note: As customer A purchased two items simultaneously, it's assumed both are to be included.

In [32]:
sql = ('''
       WITH after_join 
        	AS (SELECT *,
					RANK() OVER(PARTITION BY sales.customer_id ORDER BY order_date DESC) AS ranking
				FROM members
				LEFT JOIN sales
					ON sales.customer_id = members.customer_id
				LEFT JOIN menu
					ON sales.product_id = menu.product_id
				WHERE order_date < join_date
				)

		SELECT customer_id,
			product_name
		FROM after_join
		WHERE ranking = 1
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id product_name
0           A        sushi
1           A        curry
2           B        sushi


### Question 8: What is the total items and amount spent for each member before they became a member?

In [33]:
sql = ('''
        WITH after_join 
            AS (SELECT sales.customer_id, product_name, price
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date < (SELECT join_date FROM members)
                )

        SELECT customer_id,
               SUM(price) AS total_spend
        FROM after_join
        GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  total_spend
0           A           25
1           B           40


### Question 9: If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

Note: It's assumed that only members receive points, and only from all purchases from their joining date onwards.

In [34]:
sql = ('''
        WITH after_join 
		    AS (SELECT sales.customer_id,
                       price,
                       sales.product_id
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date >= join_date
                )

SELECT customer_id,
	   SUM(CASE
		   WHEN product_id = 1 THEN price * 20
		   ELSE price * 10
		   END) AS points
FROM after_join
GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  points
0           A     510
1           B     440


### Question 10: In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
Note: Again, this assumes that points will only be generated after the customer has become a member.

In [35]:
sql = ('''
        WITH after_join 
		    AS (SELECT sales.customer_id,
                       order_date,
                       join_date,
                       price,
                       sales.product_id
                FROM members
                LEFT JOIN sales
                    ON sales.customer_id = members.customer_id
                LEFT JOIN menu
                    ON sales.product_id = menu.product_id
                WHERE order_date >= join_date
                )

SELECT customer_id,
	   SUM(CASE
		   WHEN order_date <= DATE(join_date, '+6 day') THEN price * 20
		   WHEN product_id = 1 THEN price * 20
		   ELSE price * 10
		   END) AS points
FROM after_join
WHERE order_date <= '2021-01-31'
GROUP BY customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

  customer_id  points
0           A    1020
1           B     320


### Bonus Question 1: Join all the things

In [36]:
sql = ('''
        SELECT sales.customer_id,
	           order_date,
	           product_name,
	           price,
               CASE 
                 WHEN order_date >= join_date THEN 'Y'
                 ELSE 'N'
               END AS member
        FROM sales
        LEFT JOIN menu
            ON sales.product_id = menu.product_id
        LEFT JOIN members
            ON sales.customer_id = members.customer_id
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

   customer_id  order_date product_name  price member
0            A  2021-01-01        sushi     10      N
1            A  2021-01-01        curry     15      N
2            A  2021-01-07        curry     15      Y
3            A  2021-01-10        ramen     12      Y
4            A  2021-01-11        ramen     12      Y
5            A  2021-01-11        ramen     12      Y
6            B  2021-01-01        curry     15      N
7            B  2021-01-02        curry     15      N
8            B  2021-01-04        sushi     10      N
9            B  2021-01-11        sushi     10      Y
10           B  2021-01-16        ramen     12      Y
11           B  2021-02-01        ramen     12      Y
12           C  2021-01-01        ramen     12      N
13           C  2021-01-01        ramen     12      N
14           C  2021-01-07        ramen     12      N


### Bonus Question 2: Rank all the things

In [37]:
sql = ('''
        WITH pre_rank AS
            (SELECT sales.customer_id,
                    order_date,
                    product_name,
                    price,
                    CASE 
                      WHEN order_date >= join_date THEN 'Y'
                      ELSE 'N'
                    END AS member
            FROM sales
            LEFT JOIN menu
                ON sales.product_id = menu.product_id
            LEFT JOIN members
                ON sales.customer_id = members.customer_id
            )

        SELECT customer_id,
            order_date,
            product_name,
            price,
            member,
            CASE
              WHEN member = 'N' THEN NULL
              ELSE RANK() OVER(PARTITION BY customer_id, member ORDER BY order_date)
            END AS ranking
        FROM pre_rank
       ''')

df = pd.read_sql_query(sql, conn)
print(df)

   customer_id  order_date product_name  price member  ranking
0            A  2021-01-01        sushi     10      N      NaN
1            A  2021-01-01        curry     15      N      NaN
2            A  2021-01-07        curry     15      Y      1.0
3            A  2021-01-10        ramen     12      Y      2.0
4            A  2021-01-11        ramen     12      Y      3.0
5            A  2021-01-11        ramen     12      Y      3.0
6            B  2021-01-01        curry     15      N      NaN
7            B  2021-01-02        curry     15      N      NaN
8            B  2021-01-04        sushi     10      N      NaN
9            B  2021-01-11        sushi     10      Y      1.0
10           B  2021-01-16        ramen     12      Y      2.0
11           B  2021-02-01        ramen     12      Y      3.0
12           C  2021-01-01        ramen     12      N      NaN
13           C  2021-01-01        ramen     12      N      NaN
14           C  2021-01-07        ramen     12      N  